<h1 style="font-size:300%; font-family:cursive; background:#ff6666; color:white; text-align:center; border:10px solid ; padding:25px;">PIMA DIABETES - EDA | MODEL BUILDING </h1>

<center><img src="https://www.purdue.edu/hr/CHL/no-cost_Wellness/images/diabetes-banner.jpg"></center>

<h2 style="font-size:250%; font-family:cursive; color:#ff6666;"><b>Problem Statement:</b><h2>
    
<p style="font-size:100%; font-family:Geneva">Diabetes is one of the deadliest diseases in the world. It is not only a disease but also creator of different kinds of diseases like heart attack, blindness etc. The normal identifying process is that patients need to visit a diagnostic center, consult their doctor, and sit tight for a day or more to get their reports.</p>
<p style="font-size:100%; font-family:Geneva">So, the objective of this project is to identify whether the patient has diabetes or not based on diagnostic measurements.</p>
    
<h2 style="font-size:250%; font-family:cursive; color:#ff6666;"><b>What is Diabetes ?</b><h2>
<ul>
    <li style="font-size:100%; font-family:Geneva">Diabetes is a group of diseases in which the body doesn’t produce enough or any insulin, doesn’t properly use the insulin that is produced, or exhibits a combination of both. When any of these things happens, the body is unable to get sugar from the blood into the cells. That leads to high blood sugar levels.</li>
    <br>
    <li style="font-size:100%; font-family:Geneva">Glucose, the form of sugar found in your blood, is one of your main energy sources. A lack of insulin or resistance to insulin causes sugar to build up in your blood. This can lead to many health problems.</li>
    <br>
    <li style="font-size:100%; font-family:Geneva">The three main types of diabetes are: <b>type 1 diabetes, type 2 diabetes, gestational diabetes</b>.</li>
</ul>
    
<h2 style="font-size:250%; font-family:cursive; color:#ff6666;"><b>About Dataset:</b><h2>    
<ul>
    <li style="font-size:100%; font-family:Geneva"><b>Pregnancies:</b> No. of times pregnant</li>
    <li style="font-size:100%; font-family:Geneva"><b>Glucose:</b> Plasma Glucose Concentration (mg/dl)</li>
    <li style="font-size:100%; font-family:Geneva"><b>Blood Pressure:</b> Diastolic Blood Pressure(mmHg)</li>
    <li style="font-size:100%; font-family:Geneva"><b>Skin Thickness:</b>A value used to estimate body fat. Normal Triceps SkinFold Thickness in women is 23mm. Higher thickness leads to obesity and chances of diabetes increases.</li>
    <li style="font-size:100%; font-family:Geneva"><b>Insulin:</b> 2-Hour Serum Insulin (mu U/ml)</li>
    <li style="font-size:100%; font-family:Geneva"><b>BMI:</b> Body Mass Index (weight in kg/ height in m2)</li>
    <li style="font-size:100%; font-family:Geneva"><b>Diabetes Pedigree Function:</b>  It provides information about diabetes history in relatives and genetic relationship of those relatives with patients. Higher Pedigree Function means patient is more likely to have diabetes.</li>
    <li style="font-size:100%; font-family:Geneva"><b>Age:</b>Age (years)</li>
    <li style="font-size:100%; font-family:Geneva"><b>Outcome:</b> Class Variable (0 or 1) where ‘0’ denotes patient is not having diabetes and ‘1’ denotes patient having diabetes.</li>
</ul>


<h1 style="font-size:300%; font-family:cursive; background:#ff6666; color:white; text-align:center; border:10px solid; padding:25px;">IMPLEMENTATION USING PYTHON</h1>

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>1. Import Libraries & Load Dataset</b></h3></center>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
df.head()

<ul>
    <li style="font-size:180%; font-family:Geneva;">There are some values in Insulin, that cannot be zero. So, need to handle them by imputing.</li>
</ul>

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>2. Descriptive Statistics of Data</b></h3></center>

In [ ]:
# see the column names and its datatypes
df.info()

In [ ]:
#shape:
df.shape

In [ ]:
df.describe()

<ul>
    <li style="font-size:180%; font-family:Geneva;">There is a huge variation in mean, and we can see there's no missing values, but for some of the columns like Glucose , BP, Skin Thickness,BMI has 0 as min value, which is not possible, hence we can treat this as missingvalues and impute accordingly.</li>
</ul>

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>3. Handle the Columns with value '0'</b></h3></center>

In [ ]:
features = df.columns
cols = (df[features] == 0).sum()
print(cols)

<ul>
    <li style="font-size:180%; font-family:Geneva;">We cannot drop these values, as our data is very small. So let's handle them.</li>
</ul>

In [ ]:
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [ ]:
df.isnull().sum()

In [ ]:
msno.matrix(df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>3.1 Handle: Glucose, BloodPressure, BMI</b><h4>

In [ ]:
#Replace the null values with the median of that column:

df['Glucose'].fillna(df['Glucose'].median(), inplace =True)

df['BloodPressure'].fillna(df['BloodPressure'].median(), inplace =True)

df['BMI'].fillna(df['BMI'].median(), inplace =True)

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>3.2 Handle: Insulin based on Glucose</b><h4>

In [ ]:
by_Glucose_Age_Insulin_Grp = df.groupby(['Glucose'])

def fill_Insulin(series):
    return series.fillna(series.median())
df['Insulin'] = by_Glucose_Age_Insulin_Grp['Insulin'].transform(fill_Insulin)

In [ ]:
df['Insulin'] = df['Insulin'].fillna(df['Insulin'].mean())

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>3.3 Skinthickness with respect to BMI</b><h4>

In [ ]:
by_BMI_Insulin = df.groupby(['BMI'])

def fill_Skinthickness(series):
    return series.fillna(series.mean())
df['SkinThickness'] = by_BMI_Insulin['SkinThickness'].transform(fill_Skinthickness)

In [ ]:
df['SkinThickness'].fillna(df['SkinThickness'].mean(),inplace= True)

In [ ]:
df.isnull().sum()

<p style="font-size:180%; font-family:Geneva; text-align:center;"><b>"MISSING VALUES ARE HANDLED"</b></p>

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>4. Visualization of Target Variable</b></h3></center>

In [ ]:
import matplotlib.style as style
style.available

style.use('seaborn-pastel')
labels = ["Healthy", "Diabetic"]
df['Outcome'].value_counts().plot(kind='pie',labels=labels, subplots=True,autopct='%1.0f%%', labeldistance=1.2, figsize=(9,9))

In [ ]:
from matplotlib.pyplot import figure, show

figure(figsize=(8,6))
ax = sns.countplot(x=df['Outcome'], data=df,palette="husl")
ax.set_xticklabels(["Healthy","Diabetic"])
healthy, diabetics = df['Outcome'].value_counts().values
print("Samples of diabetic people: ", diabetics)
print("Samples of healthy people: ", healthy)

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>5. Distribution of other features w.r.t Outcome</b></h3></center>

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.1 Distribution of Pregnancies</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['Pregnancies'][df.Outcome == 1], color ="darkturquoise", rug = True)
sns.distplot(df['Pregnancies'][df.Outcome == 0], color ="lightcoral",rug = True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.2 Distribution of Glucose</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['Glucose'][df.Outcome == 1], color ="darkturquoise", rug = True)
sns.distplot(df['Glucose'][df.Outcome == 0], color ="lightcoral", rug = True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.3 Distribution of BloodPressure</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['BloodPressure'][df.Outcome == 1], color ="darkturquoise", rug=True)
sns.distplot(df['BloodPressure'][df.Outcome == 0], color ="lightcoral", rug=True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.4 Distribution of SkinThickness</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['SkinThickness'][df.Outcome == 1], color ="darkturquoise", rug=True)
sns.distplot(df['SkinThickness'][df.Outcome == 0], color ="lightcoral", rug=True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.5 Distribution of Insulin</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['Insulin'][df.Outcome == 1], color ="darkturquoise", rug=True)
sns.distplot(df['Insulin'][df.Outcome == 0], color ="lightcoral", rug=True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.6 Distribution of BMI</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['BMI'][df.Outcome == 1], color ="darkturquoise", rug=True)
sns.distplot(df['BMI'][df.Outcome == 0], color ="lightcoral", rug=True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.7 Distribution of DiabetesPedigreeFunction</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['DiabetesPedigreeFunction'][df.Outcome == 1], color ="darkturquoise", rug=True)
sns.distplot(df['DiabetesPedigreeFunction'][df.Outcome == 0], color ="lightcoral", rug=True)
plt.legend(['Diabetes', 'No Diabetes'])

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>5.8 Distribution of Age</b><h4>

In [ ]:
plt.figure()
ax = sns.distplot(df['Age'][df.Outcome == 1], color ="darkturquoise", rug=True)
sns.distplot(df['Age'][df.Outcome == 0], color ="lightcoral", rug=True)
sns.distplot(df['Age'], color ="green", rug=True)
plt.legend(['Diabetes', 'No Diabetes', 'all'])

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>6. BoxPlot to check for Outliers</b></h3></center>

In [ ]:
fig = plt.figure(figsize=(18,15))
gs = fig.add_gridspec(3,3)
gs.update(wspace=0.5, hspace=0.25)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])
ax2 = fig.add_subplot(gs[0,2])
ax3 = fig.add_subplot(gs[1,0])
ax4 = fig.add_subplot(gs[1,1])
ax5 = fig.add_subplot(gs[1,2])
ax6 = fig.add_subplot(gs[2,0])
ax7 = fig.add_subplot(gs[2,1])
ax8 = fig.add_subplot(gs[2,2])

background_color = "#c9c9ee"
# c9c9ee
color_palette = ["#f56476","#ff8811","#ff0040","#ff7f6c","#f0f66e","#990000"]
fig.patch.set_facecolor(background_color) 
ax0.set_facecolor(background_color) 
ax1.set_facecolor(background_color)
ax2.set_facecolor(background_color)
ax3.set_facecolor(background_color)
ax4.set_facecolor(background_color)
ax5.set_facecolor(background_color)
ax6.set_facecolor(background_color)
ax7.set_facecolor(background_color)
ax8.set_facecolor(background_color)

# Title of the plot
ax0.spines["bottom"].set_visible(False)
ax0.spines["left"].set_visible(False)
ax0.spines["top"].set_visible(False)
ax0.spines["right"].set_visible(False)
ax0.tick_params(left=False, bottom=False)
ax0.set_xticklabels([])
ax0.set_yticklabels([])
ax0.text(0.5,0.5,
         'Boxenplot plot for various\n features\n_________________\n\n CREDIT: Naman Manchanda',
         horizontalalignment='center',
         verticalalignment='center',
         fontsize=18, fontweight='bold',
         fontfamily='serif',
         color="#000000")

# Pregnancies 
ax1.text(-0.18, 19, 'Pregnancies', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax1.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax1,y=df['Pregnancies'],palette=["#f56476"],width=0.6)
ax1.set_xlabel("")
ax1.set_ylabel("")

# Glucose 
ax2.text(-0.1, 217, 'Glucose', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax2.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax2,y=df['Glucose'],palette=["#ff8811"],width=0.6)
ax2.set_xlabel("")
ax2.set_ylabel("")

# BloodPressure 
ax3.text(-0.20, 132, 'BloodPressure', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax3.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax3,y=df['BloodPressure'],palette=["#ff0040"],width=0.6)
ax3.set_xlabel("")
ax3.set_ylabel("")

# SkinThickness 
ax4.text(-.2, 110, 'SkinThickness', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax4.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax4,y=df['SkinThickness'],palette=["#ff7f6c"],width=0.6)
ax4.set_xlabel("")
ax4.set_ylabel("")

# Insulin 
ax5.text(-0.10, 900, 'Insulin', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax5.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax5,y=df['Insulin'],palette=["#f0f66e"],width=0.6)
ax5.set_xlabel("")
ax5.set_ylabel("")

# BMI 
ax6.text(-0.08, 77, 'BMI', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax6.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax6,y=df['BMI'],palette=["#990000"],width=0.6)
ax6.set_xlabel("")
ax6.set_ylabel("")

# DPF 
ax7.text(-0.065, 2.8, 'DPF', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax7.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax7,y=df['DiabetesPedigreeFunction'],palette=["#3339FF"],width=0.6)
ax7.set_xlabel("")
ax7.set_ylabel("")

# Age 
ax8.text(-0.08, 86, 'Age', fontsize=14, fontweight='bold', fontfamily='serif', color="#000000")
ax8.grid(color='#000000', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
sns.boxenplot(ax=ax8,y=df['Age'],palette=["#34495E"],width=0.6)
ax8.set_xlabel("")
ax8.set_ylabel("")



for s in ["top","right","left"]:
    ax1.spines[s].set_visible(False)
    ax2.spines[s].set_visible(False)
    ax3.spines[s].set_visible(False)
    ax4.spines[s].set_visible(False)
    ax5.spines[s].set_visible(False)
    ax6.spines[s].set_visible(False)
    ax7.spines[s].set_visible(False)
    ax8.spines[s].set_visible(False)

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>7. Correlation Matrix </b></h3></center>

In [ ]:
plt.figure(dpi = 120,figsize= (5,4))
mask = np.triu(np.ones_like(df.corr(),dtype = bool))
sns.heatmap(df.corr(),mask = mask, fmt = ".2f",annot=True,lw=1,cmap = 'plasma')
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)
plt.title('Correlation Heatmap')
plt.show()

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>8. Pair Plot</b></h3></center>

In [ ]:
sns.pairplot(df, hue="Outcome",palette="husl")

<center><h3 style="font-size:200%; font-family:cursive; color:#ff6666;"><b>9. MODEL BUILDING</b></h3></center>

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>9.1 Split the data into test & train</b></h4>
<ul>
    <li style="font-size:150%; font-family:Geneva;">The train-test split procedure is used to estimate the performance of machine learning algorithms when they are used to make predictions on data not used to train the model.</li>
</ul>


In [ ]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 0)

In [ ]:
print("Number transactions x_train dataset: ", x_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions x_test dataset: ", x_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

<h4 style="font-size:150%; font-family:cursive; color:#ff6666;"><b>9.2 Feature Scaling</b></h4>
<ul>
    <li style="font-size:150%; font-family:Geneva;">Feature Scaling is a technique to standardize the independent features present in the data in a fixed range. It is performed during the data pre-processing to handle highly varying magnitudes or values or units. If feature scaling is not done, then a machine learning algorithm tends to weigh greater values, higher and consider smaller values as the lower values, regardless of the unit of the values.</li>
    <li style="font-size:150%; font-family:Geneva;"><b>Standard Scaler</b>: It is a very effective technique which re-scales a feature value so that it has distribution with 0 mean value and variance equals to 1.</li>
</ul>


In [ ]:
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,roc_curve,accuracy_score,auc

In [ ]:
fpr,tpr,_=roc_curve(y_test,y_pred)
#calculate AUC
roc_auc=auc(fpr,tpr)
#plot of ROC curve for a specified class
plt.figure()
plt.plot(fpr,tpr,label='ROC curve(area= %2.f)' %roc_auc)
plt.plot([0,1],[0,1],'k--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.grid()
plt.show()

In [ ]:
# Creating some Tensors
X_train=torch.FloatTensor(x_train)
X_test=torch.FloatTensor(x_test)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

In [ ]:
# Creatng the model in pytorch
class ANN_Model(nn.Module):
    def __init__(self,input_features=8,hidden1=20,hidden2=20,output_features=2):
        super().__init__()
        self.fc1=nn.Linear(input_features,hidden1)
        self.fc2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,output_features)
        
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
    
        

In [ ]:
#Instantiate my ANN
torch.manual_seed(21)
model=ANN_Model()

In [ ]:
model.parameters

In [ ]:
model.parameters# Backward Propogation -- define the loss func and the optimizer 
loss_function=nn.CrossEntropyLoss() #for multi class 
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:
# trigger the forward propogation
epochs=4000
final_losses=[]
for i in range(epochs):
    i+i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if(i%500==1):
        print('Epochs number :{} and the loss is {}'.format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
#plotting the loss funtion
plt.plot(range(epochs),final_losses)
plt.ylabel('loss')
plt.xlabel('Epochs')


In [ ]:
#predictions
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())
        

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,predictions)

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(cm,annot=True)
plt.xlabel('Actual')
plt.ylabel('Predicted')

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)

In [ ]:
print(score)

In [ ]:
# save the model
torch.save(model,'diabetes.pt')

In [ ]:
model=torch.load('./diabetes.pt')

In [ ]:
model.eval()

<h1 style="font-size:200%; font-family:cursive; color:green; text-align:center;">Please give an upvote, if you like & find this kernel helpful. Thank You!!!!!</h1> 